# Исследование надежности заемщиков

Цель исследования:

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 
Входные данные от банка — статистика о платёжеспособности клиентов.

Ход исследования:

1) Обзор данных. Получим представление о таблице, разбремся с назначением столбцов.

2) Предобработка данных. Удалим дубликаты, проверим каждый столбец на аномальные значения и неявные дубликаты.

3) Ответы на поставленные вопросы.

## Обзор данных

Прочитаем данные и сохраним их в переменную 'data'. Получим первое представление о данных, выведя на экран первые 10 строк.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В наименовании столбцов проблем нет - используются только строчные буквы, слова разделены нижним подчеркиванием, названия столбцов соответствуют их содержанию. 

Обращает на себя внимание столбец 'days_employed'. В нем должно быть указано количество дней трудового стажа клиента, но вместо этого видим отрицательные значения или слишком большие значения (например, строка № 4 - 340266.072047 - если перевести в годы, то получится 932 года трудового стажа), кроме того, значения не целые. С этим нужно будет разобраться.	

In [3]:
# Узнаем размер таблицы
data.shape

(21525, 12)

Выведем общую информацию о таблице:

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В столбцах 'days_employed' и 'total_income' есть пропущенные значения, причем количество пропущенных значений совпадает. С этим тоже нужно будет разобраться. 
Столбец 'total_income' и  'days_employed' - вещественный тип данных. Для удобства необходимо будет заменить на целочисленный.
По другим столбцам вопросов нет - тип данных соответствует их назначению.

## Предобработка данных

Проверим каждый из столбцов методом value_counts() на наличие дубликатов и аномальных значений.

### Столбец 'children'

In [5]:
# Запрашиваем подсчет уникальных значений
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В 76 строках количество детей 20. В 47 строках значение -1. Это явно какая-то ошибка. В остальном похоже на правду.
Возможно, данные вносились вручную и значение '20' означает '2', а '-1' означает '1', но достоверно установить, сколько у этих клиентов детей на самом деле невозможно. 

Помним о цели исследования. Количество детей - очень важный параметр и заполнять его медианным или средним значением было бы некорректно, тем более количество строк с ошибочными данными довольно существенно. В этом случае лучше всего будет удалить из датафрейма строки с неверными значениями. Так мы не повлияем на конечный результат исследования.

In [6]:
# Перезаписываем таблицу с условием: "в столбце children значение не должно равняться 20 или -1"
data = data[(data['children'] != 20) & (data['children'] != -1)]
# Проверяем, что строки с неверными значениями удалены из датафрейма.
data['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

### Столбец 'days_employed'

Выясним, что за значения в столбце 'days_employed'. Выведем больше строк из этого стоблца.

In [7]:
data['days_employed'].head(50)

0      -8437.673028
1      -4024.803754
2      -5623.422610
3      -4124.747207
4     340266.072047
5       -926.185831
6      -2879.202052
7       -152.779569
8      -6929.865299
9      -2188.756445
10     -4171.483647
11      -792.701887
12              NaN
13     -1846.641941
14     -1844.956182
15      -972.364419
16     -1719.934226
17     -2369.999720
18    400281.136913
19    -10038.818549
20     -1311.604166
21      -253.685166
22     -1766.644138
23      -272.981385
24    338551.952911
25    363548.489348
26              NaN
27      -529.191635
28      -717.274324
29              NaN
30    335581.668515
31     -1682.083438
32     -4649.910832
33     -1548.637544
34     -4488.067031
35    394021.072184
36      -176.216688
37     -6448.810860
38      -597.881827
39      -650.587796
40     -1030.362606
41              NaN
42     -1257.496190
43     -4375.681384
44     -1362.041728
45     -1039.451741
46     -2262.712304
47     -2689.137274
48     -3341.067886
49     -1181.443228


Знак минус стоит перед каждым значением, кроме аномальных. Возможно, произошла какая-то проблема при выгрузке и знак минус проставился почти всем значениям.
Дробная часть, вероятно, означает количество часов. Например, начало трудового стажа в базе может отчитываться с 00ч 00м определенной даты, а при выгрузке время, меньшее чем полные сутки, переводится в дробную часть.
Если убрать минус и дробную часть, то данные похожи на правду. 

В столбце 'days_employed' отрицательные значения преобразуем в положительные методом abs(), а также обработаем слишком большие значения.
Но для начала надо убедиться в предположении, что аномальные значения всегда положительны.

In [8]:
data[data['days_employed'] > 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем


Отметим, что в выведенной части датафрейма неверные значения проставлены у категории "пенсионеры". Проверим это предположение.

In [9]:
data[(data['days_employed'] > 0) & ( data['income_type'] != 'пенсионер')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,337524.466835,31,среднее,1,женат / замужем,0,M,безработный,1,59956.991984,покупка жилья для сдачи
14798,0,395302.838654,45,Высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью


Предположение практически подтвердилось. Только две строки имеют положительное значение в столбце 'days_employed' и при этом 'income_type' не 'пенсионер'.

Оценка влияния трудового стажа на платежеспособнность клиентов не входит в цель исследования, поэтому можно было бы заменить неверные значения на медиану, и это не повлияло бы на результат. Однако, более правильным подходом будет трудовой стаж на нули. Для банка имеет значение трудового стажа на последнем месте работы, а не общий стаж. Поэтому для банка пенсионеры и безработные не имеют трудового стажа.

In [10]:
data.loc[(data['days_employed'] > 0), 'days_employed'] = 0

Отрицательные числа преобразуем в положительные.

In [11]:
data['days_employed'] = data['days_employed'].abs()

Теперь разберемся с пропусками в этом столбце.

Посчитаем долю количество пропусков в общем количестве строк столбца.

In [12]:
len(data[data['days_employed'].isna()]) / len(data['days_employed'])

0.10101859639286048

Немало. Просто удалиить эти строки нельзя, качество исследования может пострадать.
Можно заполнить пропуски медианным значением. Помним, что 3431 строка имела неверное значение, поэтому при расчете медианы исключим эти строки.

In [13]:
# Рассчитаем медианное значение столбца 'days_employed'
days_employed_median = data[data['days_employed'] != 0]['days_employed'].median()
days_employed_median

1630.4435908090563

In [14]:
# Заменим пропущенные значения на медиану
data['days_employed'] = data['days_employed'].fillna(days_employed_median)

In [15]:
# Проверим, исчезли ли пропущенные значения в столбце
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   days_employed     21402 non-null  float64
 2   dob_years         21402 non-null  int64  
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  total_income      19240 non-null  float64
 11  purpose           21402 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


И последнее, что сделаем с этим столбцом - для удобства восприятия преобразуем тип данных в целое число.

In [16]:
data['days_employed'] = data['days_employed'].astype(int)

### Столбец 'dob_years'

In [17]:
# Запрашиваем подсчет уникальных значений
data['dob_years'].value_counts()

35    614
40    603
41    603
34    597
38    595
42    592
33    577
39    572
31    556
36    553
29    543
44    543
48    536
30    536
37    531
43    510
50    509
32    506
49    505
28    501
45    494
27    490
52    483
56    482
47    480
54    476
46    469
58    461
53    457
57    457
51    446
59    441
55    441
26    406
60    376
25    356
61    353
62    351
63    268
24    263
64    263
23    252
65    194
66    183
22    183
67    167
21    110
0     100
68     99
69     83
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

В 100 строках значения возраста равно 0. Заменим на медианное значение. 

In [18]:
# Рассчитываем медиану в столбце 'dob_years'
dob_years_median = data['dob_years'].median()
# Присваиваем значение медианы тем строкам, в которых значение равно нулю.
data.loc[(data['dob_years'] == 0), 'dob_years'] = dob_years_median
# Из-за того, что значение медианы это вещественное число, изменился тип данных в столбце. Преобразуем обратно в целое число
data['dob_years'] = data['dob_years'].astype(int)

### Столбец 'education'

In [19]:
#  Проверяем столбец на уникальные значения
data['education'].value_counts() 

среднее                13667
высшее                  4698
СРЕДНЕЕ                  766
Среднее                  703
неоконченное высшее      665
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Названия дублируются, меняется только регистр букв. Используем метод str.lower() для обработки значений.

In [20]:
data['education'] = data['education'].str.lower()

In [21]:
# Проверяем, что дубликатов больше нет
data['education'].value_counts() 

среднее                15136
высшее                  5237
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

### Столбец 'education_id'

In [22]:
# В этом столбце проблем на первый взляд нет.
data['education_id'].value_counts()

1    15136
0     5237
2      741
3      282
4        6
Name: education_id, dtype: int64

### Столбец 'family_status'

In [23]:
data['family_status'].value_counts()

женат / замужем          12302
гражданский брак          4160
Не женат / не замужем     2799
в разводе                 1189
вдовец / вдова             952
Name: family_status, dtype: int64

Название категорий записаны строчными буквами, кроме "Не женат / не замужем". Может это и не критично, но всё-таки заменим.

In [24]:
#Меняем название категории и убеждаемся, что кроме названия ничего больше не изменилось.
data.loc[data['family_status'] == 'Не женат / не замужем', 'family_status'] = 'не женат / не замужем'
data['family_status'].value_counts()

женат / замужем          12302
гражданский брак          4160
не женат / не замужем     2799
в разводе                 1189
вдовец / вдова             952
Name: family_status, dtype: int64

### Столбец 'family_status_id'

In [25]:
data['family_status_id'].value_counts()

0    12302
1     4160
4     2799
3     1189
2      952
Name: family_status_id, dtype: int64

В этом столбце проблем не обнаружено.

### Столбец 'gender'

In [26]:
data['gender'].value_counts()

F      14154
M       7247
XNA        1
Name: gender, dtype: int64

В одной строке загадочный 'XNA'. Не повлияет на исследование, но лучше заменить.

Меняем значение в одной ячейке. Убеждаемся, что значение "XNA" исчезло.

In [27]:
data.loc[data['gender'] == 'XNA', 'gender'] = 'M'

In [28]:
data['gender'].value_counts()

F    14154
M     7248
Name: gender, dtype: int64

### Столбец 'income_type'

In [29]:
data['income_type'].value_counts()

сотрудник          11050
компаньон           5054
пенсионер           3839
госслужащий         1453
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

Неявных дубликатов нет, но может быть стоит сократить количество категорий.

Категории 'безработный', 'в декрете' и 'студент' объединим в одну - 'безработный'. Категорию 'предприниматель' трогать не будем, так как для банков это отдельный тип клиента - его нельзя отнести ни к категории наемных сотрудников, ни к безработным.

In [30]:
data.loc[data['income_type'] == 'в декрете', 'income_type'] = 'безработный'
data.loc[data['income_type'] == 'студент', 'income_type'] = 'безработный'
data['income_type'].value_counts()

сотрудник          11050
компаньон           5054
пенсионер           3839
госслужащий         1453
безработный            4
предприниматель        2
Name: income_type, dtype: int64

### Столбец 'debt'

In [31]:
# Проблем нет
data['debt'].value_counts()

0    19670
1     1732
Name: debt, dtype: int64

### Столбец 'total_income'

In [32]:
# Проверим на выбросы
display(data['total_income'].mean())
display(data['total_income'].median())
display(data['total_income'].max())
display(data['total_income'].min())

167448.7897572303

145020.80127962783

2265604.028722744

20667.26379327158

Полученные данные похожи на правду. Максимальный доход 2 265 604 руб. Довольно много, но можно поверить.

Помним, что в этом столбце есть пропущенные значения. Посчитаем их долю и заполним медианным значением.

In [33]:
len(data[data['total_income'].isna()]) / len(data['total_income']) 

0.10101859639286048

In [34]:
# Рассчитываем медианное значение
total_income_median = data['total_income'].median()
# Заполняем пустные строки медианным значением
data['total_income'] = data['total_income'].fillna(total_income_median)

Изменим тип данных на целое число для улучшения читаемости таблицы.

In [35]:
data['total_income'] = data['total_income'].astype(int)

In [36]:
# Убеждаемся, что пропущенных значений больше нет.
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21402 non-null  int64 
 1   days_employed     21402 non-null  int64 
 2   dob_years         21402 non-null  int64 
 3   education         21402 non-null  object
 4   education_id      21402 non-null  int64 
 5   family_status     21402 non-null  object
 6   family_status_id  21402 non-null  int64 
 7   gender            21402 non-null  object
 8   income_type       21402 non-null  object
 9   debt              21402 non-null  int64 
 10  total_income      21402 non-null  int64 
 11  purpose           21402 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Столбец 'total_income'

In [37]:
# Ради разнообразия посчитаем значения методом группировки
data.groupby('purpose')['purpose'].count()

purpose
автомобили                                476
автомобиль                                492
высшее образование                        447
дополнительное образование                457
жилье                                     642
заняться высшим образованием              496
заняться образованием                     412
на покупку автомобиля                     470
на покупку подержанного автомобиля        472
на покупку своего автомобиля              504
на проведение свадьбы                     772
недвижимость                              632
образование                               442
операции с жильем                         648
операции с коммерческой недвижимостью     646
операции с недвижимостью                  673
операции со своей недвижимостью           626
покупка жилой недвижимости                603
покупка жилья                             641
покупка жилья для сдачи                   651
покупка жилья для семьи                   640
покупка коммерческой недви

Названия категорий дублируют смысл друг друга, например 'на проведение свадьбы' и'свадьба' означаeт одно и то же. Необходимо сделать более общую категоризацию.

In [38]:
for elem in data['purpose']:
    if 'жиль' in elem or 'недвиж' in elem:
        data['purpose'] = data['purpose'].replace(elem, 'операции с недвижимостью')
    elif 'свадь' in elem:
        data['purpose'] = data['purpose'].replace(elem, 'проведение свадьбы')
    elif 'авто' in elem:
        data['purpose'] = data['purpose'].replace(elem, 'операции с автомобилем')
    elif 'образо' in elem:
        data['purpose'] = data['purpose'].replace(elem, 'получение образования')    

In [39]:
# убеждаемся, что цикл отработал правильно и заменил все значения
data['purpose'].value_counts()

операции с недвижимостью    10780
операции с автомобилем       4288
получение образования        3997
проведение свадьбы           2337
Name: purpose, dtype: int64

In [40]:
# Выводим первые несколько строк датафрейма, чтобы убедиться, что всё сделано правильно
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,операции с недвижимостью
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,операции с автомобилем
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,операции с недвижимостью
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,получение образования
4,0,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,проведение свадьбы


На этом обзор полученной таблицы закончим и приступим к исправлению выявленных проблем с данными.

Подсчитаем количество явных дубликатов

In [41]:
data.duplicated().sum()

408

Удалим строки с дубликатами и обновим индексы.

In [42]:
data = data.drop_duplicates().reset_index(drop=True)

In [43]:
#Проверим еще раз на наличие дубликатов.
data.duplicated().sum()

0

## Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

Добавим к таблице столбец total_income_category с категориями:
<br>
0–30000 — 'E';
<br>
30001–50000 — 'D';
<br>
50001–200000 — 'C';
<br>
200001–1000000 — 'B';
<br>
1000001 и выше — 'A'.

In [44]:
def income_category(income):
    if 0 <= income <= 30000:
        return 'E'
    elif 30001 <= income <= 50000:
        return 'D'
    elif 50001 <= income <= 200000:
        return 'C'
    elif 200001 <= income <= 1000000:
        return 'B'
    elif income >= 1000001:
        return 'A'
    
data['total_income_category'] = data['total_income'].apply(income_category)

In [45]:
# Проверяем, что столбец добавился в таблицу и функция отработала правильно
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,операции с недвижимостью,B
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,операции с автомобилем,C
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,операции с недвижимостью,C
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,получение образования,B
4,0,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,проведение свадьбы,C


Создадим два новых датафрейма со столбцами:
<br>
education_id и education — в первом,
<br>
family_status_id и family_status — во втором.

In [46]:
education = data[['education_id', 'education']]
education.drop_duplicates().reset_index(drop=True)

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [47]:
family = data[['family_status_id', 'family_status']]
family.drop_duplicates().reset_index(drop=True)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


Удалим столбцы 'education' и 'family_status' из датафрейма. При необходимости можно будет вопользоваться "словарями", которые были созданы только что.

In [48]:
data = data.drop('education', 1)
data = data.drop('family_status', 1)

In [49]:
# Проверяем, что таблицы удалены и данные готовы к исследованию.
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,операции с недвижимостью,B
1,1,4024,36,1,0,F,сотрудник,0,112080,операции с автомобилем,C
2,0,5623,33,1,0,M,сотрудник,0,145885,операции с недвижимостью,C
3,3,4124,32,1,0,M,сотрудник,0,267628,получение образования,B
4,0,0,53,1,1,F,пенсионер,0,158616,проведение свадьбы,C


## Ответы на вопросы

Для ответа на поставленные в качестве цели исследования вопросы посчитаем долю "проблемных" кредитов в зависимости от количества детей, семейного положения и уровня дохода.

Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Создадим сводную таблицу по количеству детей, в качестве агрегирующей функции укажем mean. Она поделит сумму (количество единиц, то есть "проблемных кредитов") в столбце debt на количество строк в этом столбце. Как раз то что нужно.

In [50]:
children = pd.pivot_table(data, index='children', values='debt', aggfunc='mean')

In [51]:
# переведем в проценты и округлим до одного знака после запятой
children['debt'] = (children['debt'] * 100).round(1)

In [52]:
# остортируем таблицу по убыванию
children.sort_values(by='debt', ascending=False)

,debt
children,
4,10.0
2,9.5
1,9.3
3,8.2
0,7.7
5,0.0


**Вывод 1: Очевидной зависимости между количеством детей и возвратом кредита в срок нет. Клиенты, у которых пять детей не имеют проблемных кредитов, но это может объясняться небольшой выборкой по этим клиентам.**

Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

Аналогично вычисляем долю клиентов, имеющих задолженность по кредитам в зависимости от их семейного положения.

In [53]:
family_status = pd.pivot_table(data, index='family_status_id', values='debt', aggfunc='mean')
family_status.reset_index(inplace=True)

In [54]:
# переведем в проценты и округлим до одного знака после запятой
family_status['debt'] = (family_status['debt'] * 100).round(1)

Для наглядности соединим таблицы с названием статусов и отсортируем по убыванию

In [55]:
family.merge(family_status, how='left', on='family_status_id').groupby('family_status').min().sort_values(by='debt', ascending=False)

,family_status_id,debt
family_status,,
не женат / не замужем,4,9.9
гражданский брак,1,9.4
женат / замужем,0,7.7
в разводе,3,7.1
вдовец / вдова,2,6.7


**Вывод 2: Чуть больше "проблемных" кредитов у холостых и людей, проживающих в гражданском браке, наименьшее у вдов/вдовцов**

Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [56]:
income = pd.pivot_table(data, index='total_income_category', values='debt', aggfunc='mean')

In [57]:
income['debt'] = (income['debt'] * 100).round(1)

In [58]:
income.sort_values(by='debt', ascending=False)

,debt
total_income_category,
E,9.1
C,8.7
A,8.0
B,7.1
D,6.0


**Вывод 3: Связи между уровнем дохода и возвратом кредита в срок нет. Чуть больше "проблемных" кредитов у людей с минимальным уровнем дохода, но ненамного отстают клиенты с наибольшим уровнем дохода.**

Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [59]:
purpose = pd.pivot_table(data, index='purpose', values='debt', aggfunc='mean')

In [60]:
purpose['debt'] = (purpose['debt'] * 100).round(1)

In [61]:
purpose.sort_values(by='debt', ascending=False)

,debt
purpose,
операции с автомобилем,9.4
получение образования,9.4
проведение свадьбы,8.0
операции с недвижимостью,7.4


**Вывод 4: Вероятность возврата кредита в срок выше для операций с недвижимостью и проведение свадьбы.**

## Общий вывод:

Проведенное исследование не установило четкой взаимосвязи между количеством детей у клиентов, их семейным положением, уровнем дохода и вероятностью возникновения задолженности по кредиту. Наименьшая доля "проблемных кредитов" у клиентов без детей, вдов/вдовцов, с доходом от 30 001 до 50 000, а также, которые берут кредит на проведение свадьбы и операции с недвижимостью.